In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 


In [3]:
data = pd.read_csv("HepatitisCdata.csv")
data.head()

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  615 non-null    int64  
 1   Category    615 non-null    object 
 2   Age         615 non-null    int64  
 3   Sex         615 non-null    object 
 4   ALB         614 non-null    float64
 5   ALP         597 non-null    float64
 6   ALT         614 non-null    float64
 7   AST         615 non-null    float64
 8   BIL         615 non-null    float64
 9   CHE         615 non-null    float64
 10  CHOL        605 non-null    float64
 11  CREA        615 non-null    float64
 12  GGT         615 non-null    float64
 13  PROT        614 non-null    float64
dtypes: float64(10), int64(2), object(2)
memory usage: 67.4+ KB


In [5]:
data= data.drop(["Unnamed: 0"], axis=1)

In [6]:
data.head()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [7]:
data["Sex"].unique()

array(['m', 'f'], dtype=object)

In [8]:
data.isnull().sum()

Category     0
Age          0
Sex          0
ALB          1
ALP         18
ALT          1
AST          0
BIL          0
CHE          0
CHOL        10
CREA         0
GGT          0
PROT         1
dtype: int64

In [9]:
li = list(data["Category"])
li2 = []
for i in range(len(li)) :
    if li[i] == "0=Blood Donor":
            li2.append(0)
    elif li[i] == "0s=suspect Blood Donor":
        li2.append(1)
    elif li[i] == "1=Hepatitis":
        li2.append(2)
    else :
        li2.append(3)
data["NewCat"]= li2

In [10]:
data =data.drop(["Category"], axis =1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     615 non-null    int64  
 1   Sex     615 non-null    object 
 2   ALB     614 non-null    float64
 3   ALP     597 non-null    float64
 4   ALT     614 non-null    float64
 5   AST     615 non-null    float64
 6   BIL     615 non-null    float64
 7   CHE     615 non-null    float64
 8   CHOL    605 non-null    float64
 9   CREA    615 non-null    float64
 10  GGT     615 non-null    float64
 11  PROT    614 non-null    float64
 12  NewCat  615 non-null    int64  
dtypes: float64(10), int64(2), object(1)
memory usage: 62.6+ KB


In [11]:
X = data.iloc[:,0:12].values
y = data.iloc[:, -1].values

In [12]:
X.shape

(615, 12)

In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [14]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(X[:, 0:13 ])
 
# Imputing the data    
X[:, 0:13 ] = si.transform(X[:, 0:13 ])

In [15]:
np.isnan(X.any())

False

In [16]:
y.shape

(615,)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [18]:
from sklearn.preprocessing import StandardScaler
SD=StandardScaler()
X_test=SD.fit_transform(X_test)
X_train = SD.fit_transform(X_train)

In [19]:

X_test = np.nan_to_num(X_test)
np.isnan(X_test.any())

False

In [20]:
y_train.shape

(461,)

In [21]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train, y_train)
y_predxg = xg.predict(X_test)

In [22]:
from sklearn.model_selection import cross_val_score
xgmean = cross_val_score(estimator=xg, X=X_train, y=y_train, cv =3)
print(xgmean.mean())

0.9522960699431288


In [23]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier()
cb.fit(X_train, y_train)
predcb = cb.predict(X_test)

Learning rate set to 0.076063
0:	learn: 1.2374532	total: 151ms	remaining: 2m 30s
1:	learn: 1.1229833	total: 155ms	remaining: 1m 17s
2:	learn: 1.0402350	total: 158ms	remaining: 52.7s
3:	learn: 0.9581340	total: 163ms	remaining: 40.5s
4:	learn: 0.8738206	total: 167ms	remaining: 33.3s
5:	learn: 0.7985049	total: 170ms	remaining: 28.2s
6:	learn: 0.7372179	total: 175ms	remaining: 24.8s
7:	learn: 0.6910074	total: 179ms	remaining: 22.2s
8:	learn: 0.6468592	total: 183ms	remaining: 20.2s
9:	learn: 0.6085487	total: 188ms	remaining: 18.6s
10:	learn: 0.5790997	total: 191ms	remaining: 17.2s
11:	learn: 0.5467549	total: 195ms	remaining: 16.1s
12:	learn: 0.5189632	total: 199ms	remaining: 15.1s
13:	learn: 0.4925425	total: 202ms	remaining: 14.2s
14:	learn: 0.4687261	total: 205ms	remaining: 13.4s
15:	learn: 0.4473445	total: 208ms	remaining: 12.8s
16:	learn: 0.4221049	total: 209ms	remaining: 12.1s
17:	learn: 0.4021157	total: 213ms	remaining: 11.6s
18:	learn: 0.3850337	total: 216ms	remaining: 11.2s
19:	learn

In [32]:
cbmean = cross_val_score(estimator=cb, X=X_train, y=y_train, cv =3)
print(cbmean.mean())

Learning rate set to 0.074502
0:	learn: 1.2453876	total: 3.87ms	remaining: 3.87s
1:	learn: 1.1501682	total: 6.92ms	remaining: 3.45s
2:	learn: 1.0664026	total: 9.86ms	remaining: 3.28s
3:	learn: 0.9909908	total: 12.6ms	remaining: 3.15s
4:	learn: 0.9205386	total: 16.2ms	remaining: 3.22s
5:	learn: 0.8494981	total: 19.2ms	remaining: 3.19s
6:	learn: 0.7885030	total: 22ms	remaining: 3.13s
7:	learn: 0.7372557	total: 24.8ms	remaining: 3.08s
8:	learn: 0.6943156	total: 28.1ms	remaining: 3.09s
9:	learn: 0.6574743	total: 30.9ms	remaining: 3.06s
10:	learn: 0.6194173	total: 33.5ms	remaining: 3.01s
11:	learn: 0.5870290	total: 36.1ms	remaining: 2.97s
12:	learn: 0.5597724	total: 38.4ms	remaining: 2.92s
13:	learn: 0.5262603	total: 40.9ms	remaining: 2.88s
14:	learn: 0.4979402	total: 43.5ms	remaining: 2.86s
15:	learn: 0.4762194	total: 46.3ms	remaining: 2.85s
16:	learn: 0.4522600	total: 48.9ms	remaining: 2.83s
17:	learn: 0.4287811	total: 51.4ms	remaining: 2.81s
18:	learn: 0.4118304	total: 54.1ms	remaining: 

In [24]:
y_test.shape

(154,)

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)
predrf = rf.predict(X_test)

In [26]:
rfmean = cross_val_score(estimator=rf, X=X_train, y=y_train, cv =3)
print(rfmean.mean())

0.941473559120618


In [27]:
X_train

array([[-0.86001316,  0.86001316, -0.93498676, ..., -0.26180998,
        -0.4455118 , -0.34892013],
       [-0.86001316,  0.86001316, -0.7321019 , ...,  0.11038633,
        -0.30260662,  0.36976098],
       [ 1.1627729 , -1.1627729 , -0.93498676, ..., -0.38587541,
        -0.53125491, -0.72717335],
       ...,
       [-0.86001316,  0.86001316,  1.19530433, ...,  0.07493906,
        -0.41216726, -0.00849224],
       [ 1.1627729 , -1.1627729 ,  0.07943757, ..., -0.13774454,
        -0.61223451,  0.21845969],
       [-0.86001316,  0.86001316, -0.63065946, ...,  0.07493906,
        -0.27402559,  0.78583952]])

In [28]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
preddt = dt.predict(X_test)

In [29]:
dtmean = cross_val_score(estimator=dt, X=X_train, y=y_train, cv =3)
print(dtmean.mean())

0.9218657159833631


In [30]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predlr = lr.predict(X_test)


In [31]:
lrmean = cross_val_score(estimator=lr, X=X_train, y=y_train, cv =3)
print(lrmean.mean())

0.9479670656141245
